In [1]:
%matplotlib inline

import math
import datetime
import matplotlib.pyplot as plt
import operator
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas_summary import DataFrameSummary
from isoweek import Week
from tqdm import tqdm
pd.options.display.max_columns = 50
from sklearn.externals import joblib

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error



In [3]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_stores = pd.read_csv('store.csv')
df_train = df_train.merge(df_stores, on='Store')
df_test = df_test.merge(df_stores, on='Store')

/home/adityasidharta/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_train.columns

Index([u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open',
       u'Promo', u'StateHoliday', u'SchoolHoliday', u'StoreType',
       u'Assortment', u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'PromoInterval'],
      dtype='object')

In [5]:
df_test.columns

Index([u'Id', u'Store', u'DayOfWeek', u'Date', u'Open', u'Promo',
       u'StateHoliday', u'SchoolHoliday', u'StoreType', u'Assortment',
       u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'PromoInterval'],
      dtype='object')

In [6]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [7]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,857,1,3,2015-09-16,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1713,1,2,2015-09-15,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,2569,1,1,2015-09-14,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,3425,1,7,2015-09-13,0.0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [8]:
DataFrameSummary(df_train).summary()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,1.01721e+06,NaN,NaN,1.01457e+06,693861,693861,1.01721e+06,509178,509178,NaN
mean,558.43,3.99834,NaN,5773.82,633.146,0.830107,0.381515,NaN,0.178647,NaN,NaN,5430.09,7.22287,2008.69,0.500564,23.2691,2011.75,NaN
std,321.909,1.99739,NaN,3849.93,464.412,0.375539,0.485759,NaN,0.383056,NaN,NaN,7715.32,3.21183,5.99264,0.5,14.096,1.66287,NaN
min,1,1,NaN,0,0,0,0,NaN,0,NaN,NaN,20,1,1900,0,1,2009,NaN
25%,280,2,NaN,3727,405,1,0,NaN,0,NaN,NaN,710,4,2006,0,13,2011,NaN
50%,558,4,NaN,5744,609,1,0,NaN,0,NaN,NaN,2330,8,2010,1,22,2012,NaN
75%,838,6,NaN,7856,837,1,1,NaN,0,NaN,NaN,6890,10,2013,1,37,2013,NaN
max,1115,7,NaN,41551,7388,1,1,NaN,1,NaN,NaN,75860,12,2015,1,50,2015,NaN
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1014567,693861,693861,1017209,509178,509178,509178
uniques,1115,7,942,21734,4086,2,2,5,2,4,3,654,12,23,2,24,7,3


In [9]:
DataFrameSummary(df_test).summary()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,41088,41088,41088,NaN,41077,41088,NaN,41088,NaN,NaN,40992,25872,25872,41088,23856,23856,NaN
mean,20544.5,555.9,3.97917,NaN,0.854322,0.395833,NaN,0.443487,NaN,NaN,5088.58,7.03525,2008.64,0.580607,24.4266,2011.82,NaN
std,11861.2,320.274,2.01548,NaN,0.352787,0.489035,NaN,0.496802,NaN,NaN,7225.49,3.14302,6.8624,0.493466,14.1613,1.69217,NaN
min,1,1,1,NaN,0,0,NaN,0,NaN,NaN,20,1,1900,0,1,2009,NaN
25%,10272.8,279.75,2,NaN,1,0,NaN,0,NaN,NaN,720,4,2006,0,13,2011,NaN
50%,20544.5,553.5,4,NaN,1,0,NaN,0,NaN,NaN,2425,7,2010,1,22,2012,NaN
75%,30816.2,832.25,6,NaN,1,1,NaN,1,NaN,NaN,6480,9,2012,1,37,2013,NaN
max,41088,1115,7,NaN,1,1,NaN,1,NaN,NaN,75860,12,2015,1,49,2015,NaN
counts,41088,41088,41088,41088,41077,41088,41088,41088,41088,41088,40992,25872,25872,41088,23856,23856,23856
uniques,41088,856,7,48,2,2,2,2,4,3,554,12,23,2,21,7,3


In [10]:
print df_train.shape
print df_test.shape

(1017209, 18)
(41088, 17)


In [11]:
def create_df_merge(train_df, test_df):
    df_train = train_df.copy()
    df_test = test_df.copy()
    df_train['Id'] = df_train.index
    df_test['Sales'] = 0.
    df_test['Customers'] = 0.
    df_train['TrainTest'] = 'Train'
    df_test['TrainTest'] = 'Test'
    df_train = df_train[sorted(df_train.columns)]
    df_test = df_test[sorted(df_test.columns)]
    assert df_train.columns.tolist() == df_test.columns.tolist()
    df_merge = pd.concat([df_train, df_test])
    return df_merge

In [12]:
def create_basic_features(df):
    df.Date = pd.to_datetime(df.Date)
    df["Store"] = df["Store"].astype(str)
    df["DayOfWeek"] = df["DayOfWeek"].astype(str)
    df["Year"] = df.Date.dt.year.astype(str)
    df["Month"] = df.Date.dt.month.astype(str)
    df["Week"] = df.Date.dt.week.astype(str)
    df["Day"] = df.Date.dt.day.astype(str)
    df['StateHoliday'] = df['StateHoliday'] != '0'
    df['SchoolHoliday'] = df['SchoolHoliday'] != 0
    df['Open'] = df['Open'] != 0
    df['Promo'] = df['Promo'] != 0
    df['CompetitionDistance'] = df['CompetitionDistance'].fillna(df['CompetitionDistance'].mean())
    df.loc[df['CompetitionOpenSinceYear'] < 2000, 'CompetitionOpenSinceYear'] = 2000
    df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].fillna(2000).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].fillna(1).astype(np.int32)
    df["CompetitionOpenSinceDateTime"] = pd.to_datetime(df.apply(lambda x: datetime.datetime(
    x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 1), axis=1).astype(pd.datetime))
    df["CompetitionDaysElapsed"] = df.Date.subtract(df["CompetitionOpenSinceDateTime"]).dt.days
    competition_distance = df['CompetitionDistance'].values
    competition_elapsed = df['CompetitionDaysElapsed'].values
    df['CompetitionDistance'] = np.where(competition_elapsed>= 0., competition_distance, 0.)
    df['CompetitionDaysElapsed'] = np.where(competition_elapsed >= 0., competition_elapsed, 0.)
    df['Promo2SinceYear'] = df['Promo2SinceYear'].fillna(2009).astype(np.int32)
    df['Promo2SinceWeek'] = df['Promo2SinceWeek'].fillna(1).astype(np.int32)
    df["Promo2SinceDateTime"] = pd.to_datetime(df.apply(lambda x: Week(
    x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2SinceDaysElapsed"] = df.Date.subtract(df["Promo2SinceDateTime"]).dt.days
    return df

In [13]:
def create_store_dictionary(merge_df):
    df = merge_df.copy()
    df = df.sort_values(['Date'])
    dict_store = {}
    for store in tqdm(np.unique(df.Store)):
        df_store = df[df.Store == store]
        sales_mean = df.loc[(df.Sales > 0.0) & (df.Store == store),"Sales"].mean()
        customer_mean = df.loc[(df.Customers > 0.0) & (df.Store == store), "Customers"].mean()
        is_open_sunday = df.loc[(df.Sales > 0.0) & (df.DayOfWeek == 7) & (df.Store == store),"Customers"].empty
        state_hols_array = df_store.StateHoliday.values
        state_hols_starting_date = df_store.Date[np.append([state_hols_array[0] == 1], (state_hols_array[:-1] == 0) & (state_hols_array[1:] == 1))].values
        school_hols_array = df_store.SchoolHoliday.values
        school_hols_starting_date = df_store.Date[np.append([school_hols_array[0] == 1], (school_hols_array[:-1] == 0) & (school_hols_array[1:] == 1))].values
        promo_array = df_store.Promo.values
        promo_starting_date = df_store.Date[np.append([promo_array[0] == 1], (promo_array[:-1] == 0) & (promo_array[1:] == 1))].values
        dict_store[store] = {
            'sales_mean' : sales_mean,
            'customer_mean' : customer_mean,
            'is_open_sunday' : is_open_sunday,
            'state_hols_array' : state_hols_starting_date,
            'school_hols_array' : school_hols_starting_date,
            'promo_array' : promo_starting_date
        }
    return dict_store

In [14]:
def get_sales_mean(dict_store, array_store):
    array_sales_mean = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        array_sales_mean[idx] = dict_store[array_store[idx]]['sales_mean']
    return array_sales_mean

def get_customer_mean(dict_store, array_store):
    customer_sales_mean = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        customer_sales_mean[idx] = dict_store[array_store[idx]]['customer_mean']
    return customer_sales_mean

def get_is_open_sunday(dict_store, array_store):
    is_open_sales_mean = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        is_open_sales_mean[idx] = dict_store[array_store[idx]]['is_open_sunday']
    return is_open_sales_mean

def before_state_hols_date(dict_store, array_date, array_store, array_state_hols):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_state_hols[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['state_hols_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            before_values = np.abs(value[np.where(value <= 0.0)])
            if len(before_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(before_values)
    return result

def after_state_hols_date(dict_store, array_date, array_store, array_state_hols):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_state_hols[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['state_hols_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            after_values = value[np.where(value >= 0.0)]
            if len(after_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(after_values)
    return result

def before_school_hols_date(dict_store, array_date, array_store, array_school_hols):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_school_hols[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['school_hols_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            before_values = np.abs(value[np.where(value <= 0.0)])
            if len(before_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(before_values)
    return result

def after_school_hols_date(dict_store, array_date, array_store, array_school_hols):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_school_hols[idx] == True:
            result[idx] = 0 
        else:
            store_array_date = dict_store[array_store[idx]]['school_hols_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            after_values = value[np.where(value >= 0.0)]
            if len(after_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(after_values)
    return result

def before_promo_date(dict_store, array_date, array_store, array_promo):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_promo[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['promo_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            before_values = np.abs(value[np.where(value <= 0.0)])
            if len(before_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(before_values)
    return result

def after_promo_date(dict_store, array_date, array_store, array_promo):
    result = np.zeros(len(array_store))
    for idx in range(len(array_store)):
        if array_promo[idx] == True:
            result[idx] = 0
        else:
            store_array_date = dict_store[array_store[idx]]['promo_array'].astype('datetime64[D]')
            value = store_array_date - np.datetime64(array_date[idx], 'D')
            value = value.astype(float)
            after_values = value[np.where(value >= 0.0)]
            if len(after_values) == 0:
                result[idx] = 0
            else:
                result[idx] = np.min(after_values)
    return result

In [15]:
def create_advanced_features(df, dict_store):
    array_store = df['Store'].values
    array_date = df['Date'].values
    array_state_hols = df['StateHoliday'].values
    array_school_hols = df['SchoolHoliday'].values
    array_promo = df['Promo'].values
    df['sales_mean'] = get_sales_mean(dict_store, array_store)
    df['customer_mean'] = get_customer_mean(dict_store, array_store)
    df['is_open_sunday'] = get_is_open_sunday(dict_store, array_store)
    df['prev_StateHoliday'] = before_state_hols_date(dict_store, array_date, array_store, array_state_hols)
    df['next_StateHoliday'] = after_state_hols_date(dict_store, array_date, array_store, array_state_hols)
    df['prev_SchoolHoliday'] = before_school_hols_date(dict_store, array_date, array_store, array_school_hols)
    df['next_SchoolHoliday'] = after_school_hols_date(dict_store, array_date, array_store, array_school_hols)
    df['prev_Promo'] = before_promo_date(dict_store, array_date, array_store, array_promo)
    df['next_Promo'] = after_promo_date(dict_store, array_date, array_store, array_promo)
    return df

In [16]:
def create_train_test_split(df, one_hot = True):
    all_features = ['DayOfWeek', 'Open',
                   u'Promo', u'StateHoliday', u'SchoolHoliday', u'StoreType',
       u'Assortment', u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'Year', u'Month', u'Week',
       u'Day', u'CompetitionDaysElapsed',
        u'Promo2SinceDaysElapsed', u'sales_mean',
       u'customer_mean', u'is_open_sunday', u'prev_StateHoliday',
       u'next_StateHoliday', u'prev_SchoolHoliday', u'next_SchoolHoliday',
       u'prev_Promo', u'next_Promo', 'TrainTest', 'Sales']
    cat_features = ['DayOfWeek', 'StoreType', 'Assortment', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
                   'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'Week', 'Day']
    for x in cat_features:
        df[x] = df[x].astype(str)
    df = df[all_features]
    if one_hot:
        df = pd.get_dummies(df, prefix = cat_features, columns= cat_features)
    else:
        df = df
    df_train = df[df['TrainTest'] == 'Train']
    df_test  = df[df['TrainTest'] == 'Test']
    
    df_X_train =  df_train.drop(['TrainTest', 'Sales'], axis = 1)
    df_X_test = df_test.drop(['TrainTest', 'Sales'], axis = 1)
    
    assert df_X_train.columns.tolist() == df_X_test.columns.tolist()
    
    X_train = df_X_train.values
    X_test = df_X_test.values
    Y_train = df_train['Sales'].values
    
    return X_train, Y_train, X_test, df_X_train.columns

In [17]:
def run_preprocessing_pipeline(df_train, df_test, one_hot = True):
    df_merge = create_df_merge(df_train, df_test)
    df_merge_basic_feat = create_basic_features(df_merge)
    dict_store = create_store_dictionary(df_merge)
    df_merge_adv_feat = create_advanced_features(df_merge, df_store)
    X_train, Y_train, X_test, column_names = create_train_test_split(df_features, one_hot)
    return  X_train, Y_train, X_test, column_names


In [18]:
df_merge = create_df_merge(df_train, df_test)

In [18]:
df_merge_basic_feat = create_basic_features(df_merge)

In [57]:
dict_store = create_store_dictionary(df_merge)

In [69]:
df_merge_adv_feat = create_advanced_features(df_merge, dict_store)

In [97]:
X_train, Y_train, X_test, column_names = create_train_test_split(df_merge_adv_feat, one_hot = True)

In [31]:
print X_train.shape
print Y_train.shape
print X_test.shape

(1017209, 188)
(1017209,)
(41088, 188)


In [99]:
joblib.dump(df_merge_basic_feat, 'df_merge_basic_feat.pkl') 
joblib.dump(dict_store, 'dict_store.pkl') 
joblib.dump(df_merge_adv_feat, 'df_merge_adv_feat.pkl') 
joblib.dump(X_train, 'X_train.pkl') 
joblib.dump(Y_train, 'Y_train.pkl') 
joblib.dump(X_test, 'X_test.pkl')
joblib.dump(column_names, 'column_names.pkl')

['column_names.pkl']

In [4]:
df_merge_basic_feat = joblib.load('df_merge_basic_feat.pkl')
dict_store = joblib.load('dict_store.pkl')
df_merge_adv_feat = joblib.load('df_merge_adv_feat.pkl')
X_full_train = joblib.load('X_train.pkl')
Y_full_train = joblib.load('Y_train.pkl')
X_test = joblib.load('X_test.pkl')
column_names = joblib.load('column_names.pkl')

In [5]:
X_train, X_val, Y_train, Y_val = train_test_split(X_full_train, Y_full_train, test_size = 0.20, random_state = 1234)

In [6]:
column_names.tolist()

['Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'CompetitionDistance',
 'Promo2',
 'CompetitionDaysElapsed',
 'Promo2SinceDaysElapsed',
 'sales_mean',
 'customer_mean',
 'is_open_sunday',
 'prev_StateHoliday',
 'next_StateHoliday',
 'prev_SchoolHoliday',
 'next_SchoolHoliday',
 'prev_Promo',
 'next_Promo',
 'DayOfWeek_1',
 'DayOfWeek_2',
 'DayOfWeek_3',
 'DayOfWeek_4',
 'DayOfWeek_5',
 'DayOfWeek_6',
 'DayOfWeek_7',
 'StoreType_a',
 'StoreType_b',
 'StoreType_c',
 'StoreType_d',
 'Assortment_a',
 'Assortment_b',
 'Assortment_c',
 'CompetitionOpenSinceMonth_1',
 'CompetitionOpenSinceMonth_10',
 'CompetitionOpenSinceMonth_11',
 'CompetitionOpenSinceMonth_12',
 'CompetitionOpenSinceMonth_2',
 'CompetitionOpenSinceMonth_3',
 'CompetitionOpenSinceMonth_4',
 'CompetitionOpenSinceMonth_5',
 'CompetitionOpenSinceMonth_6',
 'CompetitionOpenSinceMonth_7',
 'CompetitionOpenSinceMonth_8',
 'CompetitionOpenSinceMonth_9',
 'CompetitionOpenSinceYear_2000',
 'CompetitionOpenSinceYear_2001',
 'C

In [7]:
df_merge_adv_feat.head()

,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Customers,Date,DayOfWeek,Id,Open,Promo,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,SchoolHoliday,StateHoliday,Store,StoreType,TrainTest,Year,Month,Week,Day,CompetitionOpenSinceDateTime,CompetitionDaysElapsed,Promo2SinceDateTime,Promo2SinceDaysElapsed,sales_mean,customer_mean,is_open_sunday,prev_StateHoliday,next_StateHoliday,prev_SchoolHoliday,next_SchoolHoliday,prev_Promo,next_Promo
0,a,1270.0,9,2008,555.0,2015-07-31,5,0,True,True,0,1,2009,NaN,5263.0,True,False,1,c,Train,2015,7,31,31,2008-09-01,2524.0,2008-12-29,2405,4759.096031,564.049936,1.0,57.0,0.0,0.0,0.0,0.0,0.0
1,a,1270.0,9,2008,546.0,2015-07-30,4,1,True,True,0,1,2009,NaN,5020.0,True,False,1,c,Train,2015,7,31,30,2008-09-01,2523.0,2008-12-29,2404,4759.096031,564.049936,1.0,56.0,0.0,0.0,0.0,0.0,0.0
2,a,1270.0,9,2008,523.0,2015-07-29,3,2,True,True,0,1,2009,NaN,4782.0,True,False,1,c,Train,2015,7,31,29,2008-09-01,2522.0,2008-12-29,2403,4759.096031,564.049936,1.0,55.0,0.0,0.0,0.0,0.0,0.0
3,a,1270.0,9,2008,560.0,2015-07-28,2,3,True,True,0,1,2009,NaN,5011.0,True,False,1,c,Train,2015,7,31,28,2008-09-01,2521.0,2008-12-29,2402,4759.096031,564.049936,1.0,54.0,0.0,0.0,0.0,0.0,0.0
4,a,1270.0,9,2008,612.0,2015-07-27,1,4,True,True,0,1,2009,NaN,6102.0,True,False,1,c,Train,2015,7,31,27,2008-09-01,2520.0,2008-12-29,2401,4759.096031,564.049936,1.0,53.0,0.0,0.0,0.0,0.0,0.0


In [8]:
DataFrameSummary(df_merge_adv_feat).summary()

,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Customers,Date,DayOfWeek,Id,Open,Promo,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,SchoolHoliday,StateHoliday,Store,StoreType,TrainTest,Year,Month,Week,Day,CompetitionOpenSinceDateTime,CompetitionDaysElapsed,Promo2SinceDateTime,Promo2SinceDaysElapsed,sales_mean,customer_mean,is_open_sunday,prev_StateHoliday,next_StateHoliday,prev_SchoolHoliday,next_SchoolHoliday,prev_Promo,next_Promo
count,NaN,1.0583e+06,NaN,NaN,1.0583e+06,NaN,NaN,1.0583e+06,NaN,NaN,1.0583e+06,NaN,NaN,NaN,1.0583e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0583e+06,NaN,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06,1.0583e+06
mean,NaN,4977.7,NaN,NaN,608.564,NaN,NaN,489655,NaN,NaN,0.503671,NaN,NaN,NaN,5549.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2902.97,NaN,1361.48,6940.16,757.689,1,30.6031,24.3145,26.1157,22.7906,5.65169,3.10491
std,NaN,7516.57,NaN,NaN,471.449,NaN,NaN,302941,NaN,NaN,0.499987,NaN,NaN,NaN,3935.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.87,NaN,773.68,2387,355.863,0,31.0979,29.8057,23.55,23.0989,6.83561,3.32256
min,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-881,2703.74,240.183,1,0,0,0,0,0,0
25%,NaN,450,NaN,NaN,363,NaN,NaN,223486,NaN,NaN,0,NaN,NaN,NaN,3319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,928,NaN,775,5321.14,543.676,1,4,1,6,2,0,0
50%,NaN,2060,NaN,NaN,594,NaN,NaN,488060,NaN,NaN,1,NaN,NaN,NaN,5600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2810,NaN,1576,6607.94,681.15,1,20,12,22,18,6,2
75%,NaN,6250,NaN,NaN,824,NaN,NaN,752634,NaN,NaN,1,NaN,NaN,NaN,7749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5004,NaN,1972,7967.27,867.466,1,52,38,41,37,10,6
max,NaN,75860,NaN,NaN,7388,NaN,NaN,1.01721e+06,NaN,NaN,1,NaN,NaN,NaN,41551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5738,NaN,2453,21757.5,3403.46,1,135,195,118,194,188,16
counts,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,533034,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297,1058297
uniques,3,655,12,16,4086,990,7,1017209,2,2,2,24,7,3,21734,2,2,1115,4,2,3,12,52,31,162,5739,56,3335,1115,1114,1,136,147,119,118,21,17


In [9]:
from sklearn.metrics import make_scorer

def rmpse_loss_func(ground_truth, predictions):
    idx = np.where(ground_truth > 0.)
    ground_truth = ground_truth[idx]
    predictions = predictions[idx]
    err = np.sqrt(np.mean((np.true_divide(predictions, ground_truth) - 1.)**2))
    return err

rmpse_loss  = make_scorer(rmpse_loss_func, greater_is_better=False)

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from keras.models import load_model
from keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from keras.models import Model
from keras import backend as K
from xgboost import XGBRegressor

## Linear Regression

In [242]:
linear_model = LinearRegression().fit(X_train, Y_train)

# NN 

In [278]:
NN_idx = np.where(Y_train != 0.)
X_train_NN = X_train[NN_idx]
Y_train_NN = Y_train[NN_idx]

In [279]:
def rmspe_NN(y_true, y_pred):
    err = K.sqrt(K.mean(((y_pred / y_true) - 1.)**2))
    return err

In [286]:
inputs = Input(shape = (X_train_NN.shape[1],), name='Input')
X = Dense(2048, name='Dense1', activation='relu')(inputs)
X = BatchNormalization(name='BN1')(X)
X = Dropout(rate= 0.25)(X)
X = Dense(1024, name='Dense2', activation='relu')(X)
X = BatchNormalization(name='BN2')(X)
X = Dropout(rate= 0.20)(X)
X = Dense(512, name='Dense3', activation='relu')(X)
X = BatchNormalization(name='BN3')(X)
X = Dropout(rate= 0.15)(X)
X = Dense(256, name='Dense4', activation='relu')(X)
X = BatchNormalization(name='BN4')(X)
X = Dropout(rate= 0.10)(X)
X = Dense(128, name='Dense5', activation='relu')(X)
X = BatchNormalization(name='BN5')(X)
X = Dropout(rate= 0.05)(X)
outputs = Dense(1, name='Dense6', activation='relu')(X)

In [287]:
NN_model = Model(inputs=inputs, outputs=outputs, name='NN_Model')
NN_model.compile('adam', 'mean_squared_error' , metrics=[rmspe_NN])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 188)               0         
_________________________________________________________________
Dense1 (Dense)               (None, 2048)              387072    
_________________________________________________________________
BN1 (BatchNormalization)     (None, 2048)              8192      
_________________________________________________________________
dropout_31 (Dropout)         (None, 2048)              0         
_________________________________________________________________
Dense2 (Dense)               (None, 1024)              2098176   
_________________________________________________________________
BN2 (BatchNormalization)     (None, 1024)              4096      
_________________________________________________________________
dropout_32 (Dropout)         (None, 1024)              0         
__________

In [289]:
NN_model.fit(X_train_NN, Y_train_NN, epochs=5, batch_size=64)

Epoch 1/3
675490/675490 [==============================] - 534s 791us/step - loss: 2347962.6519 - rmspe_NN: 0.2605
Epoch 2/3
675490/675490 [==============================] - 530s 785us/step - loss: 2210722.0510 - rmspe_NN: 0.2567
Epoch 3/3
675490/675490 [==============================] - 531s 785us/step - loss: 2155088.0333 - rmspe_NN: 0.2547


In [ ]:
NN_model.save('NN_model.h5')

## Xgboost - SKLearn Wrapper 

In [58]:
xgb_model = XGBRegressor(objective = 'reg:linear', booster = 'gbtree', eta = 0.02, max_depth = 12, subsample = 0.9, colsample_bytree = 0.7, silent = False, n_jobs = 8)

In [59]:
xgb_model.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eta=0.02, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=12, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=8, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=False, subsample=0.9)

In [42]:
xgb_params = {
    'learning_rate' : np.arange(0.01, 0.21, 0.01),
    'max_depth' : np.arange(3, 25, 1),
    'gamma' : np.arange(0., 3., 1.),
}

In [41]:
start = time.time()
xgb_CV_model = RandomizedSearchCV(estimator=xgb_model, param_distributions=xgb_params, n_iter= 10, scoring='neg_mean_squared_error', cv=3, verbose=4, n_jobs = 1,  return_train_score = True).fit(X_train, Y_train)
end = time.time()
end - start

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1 
[CV]  gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1, score=-1058330.77212, total= 1.8min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s


[CV] gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1 
[CV]  gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1, score=-1229509.33876, total= 1.8min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.8min remaining:    0.0s


[CV] gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1 
[CV]  gamma=3.0, learning_rate=0.060000000000000005, colsample_bytree=0.9999999999999999, max_depth=6, min_child_weight=1, score=-1128630.50722, total= 1.9min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.7min remaining:    0.0s


[CV] gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1 
[CV]  gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1, score=-1912769.86368, total= 1.0min
[CV] gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1 
[CV]  gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1, score=-2143057.32365, total= 1.0min
[CV] gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1 
[CV]  gamma=1.0, learning_rate=0.03, colsample_bytree=0.9999999999999999, max_depth=3, min_child_weight=1, score=-2032364.68996, total= 1.0min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  9.1min finished


692.3348560333252

In [303]:
xgb_CV_model = joblib.load("xgb_CV_model.pkl")

## Native SKlearn

In [306]:
dtrain = xgb.DMatrix(X_train, label= Y_train)

In [312]:
dtest = xgb.DMatrix(X_val, label=Y_val)

In [317]:
def rmpse(preds, dtrain):
    labels = dtrain.get_label()
    idx = np.where(labels > 0.)
    labels = labels[idx]
    preds = preds[idx]
    err = np.sqrt(np.mean((np.true_divide(preds, labels) - 1.)**2))
    return 'error', err

In [318]:
watchlist = [(dtest, 'eval'), (dtrain, 'train')]

In [319]:
param = { 'objective' : "reg:linear", 
          'booster' : "gbtree",
          'eta'                 :0.05, 
          'max_depth'           :9, 
          'subsample'           : 0.9,
          'colsample_bytree'    : 0.7,
          'n_thread' : 8
        }

In [ ]:
bst = xgb.Booster()
bst.load_model('bst.model')

In [320]:
bst = xgb.train(param, dtrain, evals=watchlist, num_boost_round = 2000, feval= rmpse, maximize = False)

[0]	eval-rmse:6634.19	train-rmse:6629.6	eval-error:0.943625	train-error:0.943758
[1]	eval-rmse:6316.96	train-rmse:6312.4	eval-error:0.895238	train-error:0.895542
[2]	eval-rmse:6013.8	train-rmse:6009.41	eval-error:0.848785	train-error:0.849286
[3]	eval-rmse:5724.96	train-rmse:5720.79	eval-error:0.805014	train-error:0.805767
[4]	eval-rmse:5454.02	train-rmse:5449.88	eval-error:0.763989	train-error:0.765031
[5]	eval-rmse:5198.88	train-rmse:5194.77	eval-error:0.724168	train-error:0.725551
[6]	eval-rmse:4952.51	train-rmse:4948.47	eval-error:0.68725	train-error:0.689047
[7]	eval-rmse:4719.42	train-rmse:4715.43	eval-error:0.652359	train-error:0.654613
[8]	eval-rmse:4503.21	train-rmse:4499.14	eval-error:0.618927	train-error:0.621629
[9]	eval-rmse:4295.31	train-rmse:4291.18	eval-error:0.588336	train-error:0.591588
[10]	eval-rmse:4101.54	train-rmse:4097.56	eval-error:0.558605	train-error:0.562394
[11]	eval-rmse:3913.86	train-rmse:3910.07	eval-error:0.531175	train-error:0.535585
[12]	eval-rmse:374

[100]	eval-rmse:920.501	train-rmse:900.014	eval-error:0.204668	train-error:0.236846
[101]	eval-rmse:919.107	train-rmse:898.454	eval-error:0.204475	train-error:0.236643
[102]	eval-rmse:917.656	train-rmse:896.869	eval-error:0.204374	train-error:0.236572
[103]	eval-rmse:915.864	train-rmse:894.924	eval-error:0.204136	train-error:0.236343
[104]	eval-rmse:914.224	train-rmse:893.118	eval-error:0.203818	train-error:0.235675
[105]	eval-rmse:912.866	train-rmse:891.646	eval-error:0.20373	train-error:0.23525
[106]	eval-rmse:911.411	train-rmse:890.005	eval-error:0.203589	train-error:0.235018
[107]	eval-rmse:909.916	train-rmse:888.412	eval-error:0.203369	train-error:0.234769
[108]	eval-rmse:908.829	train-rmse:887.159	eval-error:0.203229	train-error:0.234317
[109]	eval-rmse:907.56	train-rmse:885.7	eval-error:0.203192	train-error:0.234144
[110]	eval-rmse:905.603	train-rmse:883.559	eval-error:0.2029	train-error:0.233531
[111]	eval-rmse:903.797	train-rmse:881.575	eval-error:0.2026	train-error:0.232866
[

[199]	eval-rmse:817.998	train-rmse:784.401	eval-error:0.18442	train-error:0.211036
[200]	eval-rmse:817.244	train-rmse:783.479	eval-error:0.184341	train-error:0.210836
[201]	eval-rmse:816.644	train-rmse:782.786	eval-error:0.184163	train-error:0.210731
[202]	eval-rmse:816.322	train-rmse:782.395	eval-error:0.184094	train-error:0.210651
[203]	eval-rmse:815.928	train-rmse:781.907	eval-error:0.184055	train-error:0.21065
[204]	eval-rmse:815.685	train-rmse:781.547	eval-error:0.184003	train-error:0.210542
[205]	eval-rmse:815.068	train-rmse:780.859	eval-error:0.183809	train-error:0.210409
[206]	eval-rmse:814.613	train-rmse:780.27	eval-error:0.18376	train-error:0.21032
[207]	eval-rmse:814.048	train-rmse:779.566	eval-error:0.183663	train-error:0.21023
[208]	eval-rmse:812.98	train-rmse:778.491	eval-error:0.183484	train-error:0.21001
[209]	eval-rmse:812.581	train-rmse:777.932	eval-error:0.183457	train-error:0.210001
[210]	eval-rmse:812.151	train-rmse:777.387	eval-error:0.183396	train-error:0.209944


[298]	eval-rmse:768.57	train-rmse:725.115	eval-error:0.17213	train-error:0.195419
[299]	eval-rmse:768.293	train-rmse:724.676	eval-error:0.172096	train-error:0.195372
[300]	eval-rmse:767.696	train-rmse:723.916	eval-error:0.171988	train-error:0.195259
[301]	eval-rmse:767.538	train-rmse:723.696	eval-error:0.171952	train-error:0.194971
[302]	eval-rmse:766.982	train-rmse:722.982	eval-error:0.171878	train-error:0.194109
[303]	eval-rmse:766.754	train-rmse:722.689	eval-error:0.171846	train-error:0.193987
[304]	eval-rmse:766.612	train-rmse:722.489	eval-error:0.17176	train-error:0.193928
[305]	eval-rmse:766.276	train-rmse:722.062	eval-error:0.17171	train-error:0.193882
[306]	eval-rmse:765.931	train-rmse:721.64	eval-error:0.171633	train-error:0.193818
[307]	eval-rmse:765.786	train-rmse:721.395	eval-error:0.171628	train-error:0.193775
[308]	eval-rmse:765.376	train-rmse:720.736	eval-error:0.17154	train-error:0.19367
[309]	eval-rmse:765.102	train-rmse:720.398	eval-error:0.171513	train-error:0.193608

[397]	eval-rmse:731.473	train-rmse:678.91	eval-error:0.161999	train-error:0.182938
[398]	eval-rmse:730.895	train-rmse:678.247	eval-error:0.161936	train-error:0.182874
[399]	eval-rmse:730.773	train-rmse:678.031	eval-error:0.161865	train-error:0.182869
[400]	eval-rmse:730.095	train-rmse:677.259	eval-error:0.161517	train-error:0.182753
[401]	eval-rmse:729.693	train-rmse:676.807	eval-error:0.161386	train-error:0.182623
[402]	eval-rmse:729.48	train-rmse:676.481	eval-error:0.16133	train-error:0.182575
[403]	eval-rmse:729.251	train-rmse:676.195	eval-error:0.161185	train-error:0.182374
[404]	eval-rmse:728.972	train-rmse:675.89	eval-error:0.161134	train-error:0.182322
[405]	eval-rmse:728.7	train-rmse:675.536	eval-error:0.161073	train-error:0.182275
[406]	eval-rmse:728.626	train-rmse:675.41	eval-error:0.16104	train-error:0.182261
[407]	eval-rmse:728.271	train-rmse:674.968	eval-error:0.160988	train-error:0.18222
[408]	eval-rmse:728.032	train-rmse:674.669	eval-error:0.160537	train-error:0.182143
[

...

[1570]	eval-rmse:600.288	train-rmse:483.832	eval-error:0.124921	train-error:0.137959
[1571]	eval-rmse:600.25	train-rmse:483.772	eval-error:0.124915	train-error:0.137953
[1572]	eval-rmse:600.187	train-rmse:483.667	eval-error:0.124905	train-error:0.137932
[1573]	eval-rmse:600.144	train-rmse:483.559	eval-error:0.124855	train-error:0.1379
[1574]	eval-rmse:600.143	train-rmse:483.492	eval-error:0.124857	train-error:0.137892
[1575]	eval-rmse:600.115	train-rmse:483.412	eval-error:0.124855	train-error:0.137888
[1576]	eval-rmse:600.009	train-rmse:483.244	eval-error:0.124837	train-error:0.137865
[1577]	eval-rmse:599.942	train-rmse:483.114	eval-error:0.124829	train-error:0.137852
[1578]	eval-rmse:599.786	train-rmse:482.956	eval-error:0.124755	train-error:0.137699
[1579]	eval-rmse:599.666	train-rmse:482.778	eval-error:0.124736	train-error:0.137682
[1580]	eval-rmse:599.629	train-rmse:482.71	eval-error:0.12473	train-error:0.137669
[1581]	eval-rmse:599.62	train-rmse:482.69	eval-error:0.124729	train-er

[1667]	eval-rmse:595.89	train-rmse:475.488	eval-error:0.123692	train-error:0.132137
[1668]	eval-rmse:595.805	train-rmse:475.402	eval-error:0.123687	train-error:0.132131
[1669]	eval-rmse:595.787	train-rmse:475.354	eval-error:0.123684	train-error:0.132126
[1670]	eval-rmse:595.748	train-rmse:475.27	eval-error:0.123678	train-error:0.13212
[1671]	eval-rmse:595.715	train-rmse:475.18	eval-error:0.123658	train-error:0.132101
[1672]	eval-rmse:595.681	train-rmse:475.105	eval-error:0.123649	train-error:0.132091
[1673]	eval-rmse:595.66	train-rmse:475.02	eval-error:0.123638	train-error:0.132071
[1674]	eval-rmse:595.626	train-rmse:474.94	eval-error:0.123636	train-error:0.132065
[1675]	eval-rmse:595.614	train-rmse:474.889	eval-error:0.123638	train-error:0.132061
[1676]	eval-rmse:595.545	train-rmse:474.767	eval-error:0.123645	train-error:0.132022
[1677]	eval-rmse:595.53	train-rmse:474.736	eval-error:0.123641	train-error:0.131997
[1678]	eval-rmse:595.44	train-rmse:474.61	eval-error:0.123631	train-error

[1764]	eval-rmse:591.406	train-rmse:466.699	eval-error:0.1228	train-error:0.127503
[1765]	eval-rmse:591.389	train-rmse:466.634	eval-error:0.12279	train-error:0.127478
[1766]	eval-rmse:591.355	train-rmse:466.561	eval-error:0.122786	train-error:0.127471
[1767]	eval-rmse:591.333	train-rmse:466.495	eval-error:0.12278	train-error:0.12746
[1768]	eval-rmse:591.248	train-rmse:466.37	eval-error:0.122746	train-error:0.127443
[1769]	eval-rmse:591.223	train-rmse:466.298	eval-error:0.122741	train-error:0.127431
[1770]	eval-rmse:591.2	train-rmse:466.244	eval-error:0.122736	train-error:0.127425
[1771]	eval-rmse:591.156	train-rmse:466.128	eval-error:0.122736	train-error:0.127418
[1772]	eval-rmse:591.112	train-rmse:466.048	eval-error:0.122718	train-error:0.127382
[1773]	eval-rmse:591.096	train-rmse:466.001	eval-error:0.122714	train-error:0.127377
[1774]	eval-rmse:591.08	train-rmse:465.955	eval-error:0.122712	train-error:0.127375
[1775]	eval-rmse:591.06	train-rmse:465.884	eval-error:0.122708	train-error

[1861]	eval-rmse:587.795	train-rmse:459.241	eval-error:0.121826	train-error:0.126133
[1862]	eval-rmse:587.779	train-rmse:459.203	eval-error:0.121817	train-error:0.12598
[1863]	eval-rmse:587.74	train-rmse:459.133	eval-error:0.12181	train-error:0.125971
[1864]	eval-rmse:587.708	train-rmse:459.069	eval-error:0.121808	train-error:0.125961
[1865]	eval-rmse:587.673	train-rmse:458.988	eval-error:0.121803	train-error:0.125952
[1866]	eval-rmse:587.635	train-rmse:458.913	eval-error:0.121794	train-error:0.125941
[1867]	eval-rmse:587.626	train-rmse:458.879	eval-error:0.121792	train-error:0.125936
[1868]	eval-rmse:587.575	train-rmse:458.774	eval-error:0.121764	train-error:0.125907
[1869]	eval-rmse:587.556	train-rmse:458.738	eval-error:0.121762	train-error:0.125902
[1870]	eval-rmse:587.531	train-rmse:458.658	eval-error:0.12176	train-error:0.125896
[1871]	eval-rmse:587.498	train-rmse:458.579	eval-error:0.121748	train-error:0.125883
[1872]	eval-rmse:587.474	train-rmse:458.508	eval-error:0.121747	train

[1958]	eval-rmse:584.565	train-rmse:452.396	eval-error:0.121092	train-error:0.125028
[1959]	eval-rmse:584.513	train-rmse:452.301	eval-error:0.121081	train-error:0.125017
[1960]	eval-rmse:584.504	train-rmse:452.228	eval-error:0.121078	train-error:0.125009
[1961]	eval-rmse:584.485	train-rmse:452.158	eval-error:0.121068	train-error:0.125001
[1962]	eval-rmse:584.426	train-rmse:452.067	eval-error:0.121045	train-error:0.124983
[1963]	eval-rmse:584.378	train-rmse:451.979	eval-error:0.121024	train-error:0.124971
[1964]	eval-rmse:584.333	train-rmse:451.897	eval-error:0.121017	train-error:0.124956
[1965]	eval-rmse:584.325	train-rmse:451.849	eval-error:0.121016	train-error:0.12495
[1966]	eval-rmse:584.286	train-rmse:451.792	eval-error:0.121008	train-error:0.124931
[1967]	eval-rmse:584.235	train-rmse:451.682	eval-error:0.120991	train-error:0.124906
[1968]	eval-rmse:584.192	train-rmse:451.631	eval-error:0.120986	train-error:0.124898
[1969]	eval-rmse:584.169	train-rmse:451.574	eval-error:0.120978	tr

In [321]:
bst.save_model('bst.model')

## Random Forest

In [3]:
rf_params = {"max_depth": [5, 9 ,12, 15, None],
              "max_features": [0.2, 'sqrt', 0.5],
              "n_estimators":[200],
            }

In [ ]:
rf_model = RandomForestRegressor(verbose=4)
rf_CV_model = GridSearchCV(estimator=rf_model, param_grid=rf_params,
                                   scoring= rmpse_loss, cv=3, n_jobs = -1, return_train_score = True).fit(X_train, Y_train)

building tree 1 of 200
building tree 2 of 200


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.9s remaining:    0.0s


building tree 3 of 200


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.6s remaining:    0.0s


building tree 2 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 1 of 200
building tree 7 of 200


...

building tree 3 of 200
building tree 107 of 200
building tree 128 of 200
building tree 136 of 200


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.3min remaining:    0.0s


building tree 4 of 200
building tree 108 of 200
building tree 5 of 200
building tree 125 of 200
building tree 129 of 200
building tree 137 of 200
building tree 129 of 200
building tree 109 of 200


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.2min remaining:    0.0s


building tree 4 of 200
building tree 138 of 200
building tree 126 of 200
building tree 130 of 200
building tree 110 of 200
building tree 5 of 200
building tree 139 of 200
building tree 6 of 200
building tree 130 of 200
building tree 111 of 200
building tree 140 of 200
building tree 5 of 200
building tree 131 of 200
building tree 127 of 200
building tree 112 of 200
building tree 141 of 200
building tree 131 of 200
building tree 6 of 200
building tree 7 of 200
building tree 113 of 200
building tree 142 of 200
building tree 128 of 200
building tree 132 of 200
building tree 6 of 200
building tree 114 of 200
building tree 132 of 200
building tree 143 of 200
building tree 7 of 200
building tree 115 of 200
building tree 133 of 200
building tree 129 of 200
building tree 8 of 200
building tree 144 of 200
building tree 133 of 200
building tree 116 of 200
building tree 7 of 200
building tree 145 of 200
building tree 134 of 200
building tree 130 of 200
building tree 117 of 200
building tree 8 of 2

In [ ]:
joblib.dump(rf_CV_model, 'rf_CV_model.pkl')

## Scoring

In [290]:
rmpse_loss(linear_model, X_val, Y_val)

-0.293745468990658

In [326]:
NN_val = rmpse_loss_func(Y_val,NN_model.predict(X_val).reshape(-1))
print -NN_val

-2686.444039362812


In [304]:
rmpse_loss(xgb_CV_model, X_val, Y_val)

-0.1384333502214052

In [325]:
bst_val = rmpse_loss_func(Y_val,bst.predict(dtest))
print -bst_val

-0.12082965255340518


In [217]:
metrics_rmpse(rf_CV_model, X_val, Y_val)

('error', 0.07537814724850081)

## Prediction

In [329]:
new_X_test = X_test[df_test.sort_values(['Id']).index]
dsubmit = xgb.DMatrix(new_X_test)
sales_value =  bst.predict(dsubmit)

In [332]:
result = {
    'Id': df_test.sort_values(['Id']).Id.values,
    'Sales':  sales_value
}

df_result = pd.DataFrame(result)

In [333]:
df_result.to_csv('entry.csv', index=False)